In [2]:
import findspark
findspark.init('/home/gerardo-rodriguez/spark-4.0.0-bin-hadoop3')

In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('create_reatings').getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
25/08/25 11:28:09 WARN Utils: Your hostname, Lanz-Lenovo, resolves to a loopback address: 127.0.1.1; using 192.168.1.145 instead (on interface wlp2s0)
25/08/25 11:28:09 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/08/25 11:28:11 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/08/25 11:28:11 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
25/08/25 11:28:11 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


In [6]:
users = spark.read.csv('netflix_users.csv', header=True, inferSchema=True)
shows = spark.read.csv('netflix_titles.csv', header=True, inferSchema=True)

In [7]:
users.printSchema()

root
 |-- User_ID: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Country: string (nullable = true)
 |-- Subscription_Type: string (nullable = true)
 |-- Watch_Time_Hours: double (nullable = true)
 |-- Favorite_Genre: string (nullable = true)
 |-- Last_Login: date (nullable = true)



In [8]:
shows = shows.select(['show_id',
                     'type',
                     'title',
                     'director',
                     'cast',
                     'country',
                     'date_added',
                     'release_year',
                     'rating',
                     'duration',
                     'listed_in',
                     'description'])

In [9]:
count_show = shows.count()
count_user = users.count()

In [10]:
import numpy as np
import random as rm
import pandas as pd
from pyspark.sql.functions import col

In [11]:
id_show = [idx for idx in range(1, count_show)]

In [12]:
def get_prob(cuantity):
    try:
        cuantity + 1
        
        list_prob = [1/cuantity for _ in range(cuantity)]
        list_index = list(range(cuantity))

        for _ in range(cuantity * 2): 
            idx = rm.choice(list_index)
            idx_2 = rm.choice([i for i in list_index if i != idx])

            val = list_prob[idx]
            if val == 0:
                continue

            rest = rm.uniform(0, val)
            total = val - rest

            list_prob[idx] = total
            list_prob[idx_2] += rest  

        s = sum(list_prob)
        list_prob = [p/s for p in list_prob]

        return list_prob

    except Exception as e:
        print("error:", e)
        return False

In [13]:
def create_map(name_1, name_2, name_3, size, rang_1, rang_2, prob_col_1=None, prob_col_2=None):
    try:

        rang_1 = [x for x in range(1, rang_1+1)]
        rang_2 = [f's{x}' for x in range(1, rang_2+1)]

        
        pdf = pd.DataFrame({
            name_1: np.random.choice(rang_1, size=size, p=prob_col_1),
            name_2: np.random.choice(rang_2, size=size, p=prob_col_2),
            name_3: [rm.uniform(0, 5) for _ in range(size)]
        })
        
        return spark.createDataFrame(pdf)

    except Exception as e:
        print(f'error: {e}')
        return False

In [14]:
prob_show = get_prob(count_show)
col_show = np.random.choice(count_show, (count_user), prob_show)

In [15]:
prob_user = get_prob(count_user)

In [31]:
df = create_map('user', 'film', 'rating', 100000, count_user, count_show, prob_user, prob_show)

In [17]:
df.printSchema()

root
 |-- user: long (nullable = true)
 |-- film: string (nullable = true)
 |-- rating: double (nullable = true)



In [18]:
shows.printSchema()

root
 |-- show_id: string (nullable = true)
 |-- type: string (nullable = true)
 |-- title: string (nullable = true)
 |-- director: string (nullable = true)
 |-- cast: string (nullable = true)
 |-- country: string (nullable = true)
 |-- date_added: string (nullable = true)
 |-- release_year: string (nullable = true)
 |-- rating: string (nullable = true)
 |-- duration: string (nullable = true)
 |-- listed_in: string (nullable = true)
 |-- description: string (nullable = true)



In [32]:
df = df.join(users, df['user'] == users['User_ID'], 'left').select('user', 'Name', 'Age', 'Country', 'film', 'rating')

df = df.join(shows, df['film'] == shows['show_id'], 'left').select('user', df['Name'], 'Age', df['Country'], 'film', 'title', 'listed_in' ,'duration', df['rating'])

In [33]:
df.printSchema()

root
 |-- user: long (nullable = true)
 |-- Name: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Country: string (nullable = true)
 |-- film: string (nullable = true)
 |-- title: string (nullable = true)
 |-- listed_in: string (nullable = true)
 |-- duration: string (nullable = true)
 |-- rating: double (nullable = true)



In [34]:
df.show()

+-----+-----------------+---+---------+-----+--------------------+--------------------+---------+------------------+
| user|             Name|Age|  Country| film|               title|           listed_in| duration|            rating|
+-----+-----------------+---+---------+-----+--------------------+--------------------+---------+------------------+
|21139|      Katie Smith| 52|   Brazil|s3920|            Thriller|       Horror Movies|   87 min|4.4975901966322525|
|24347|   Chris Martinez| 68|       UK|s4200|    Back with the Ex|International TV ...| 1 Season| 4.903189088327801|
| 5694|       Alex Smith| 74|   Brazil|s1432|         Stuck Apart|Comedies, Dramas,...|   97 min|0.6419307271545921|
| 7691|   Michael Miller| 20|      USA|s5352|      Handsome Devil|Comedies, Dramas,...|   95 min| 3.585136057965486|
|19645|    Katie Johnson| 33|   Brazil|s2656|The Silence of th...|International Mov...|   93 min|2.4153943972246323|
| 3471| Michael Williams| 18|Australia|s5060|A Futile and Stup..

Exception ignored in: <_io.BufferedWriter name=5>
Traceback (most recent call last):
  File "/home/gerardo-rodriguez/spark-4.0.0-bin-hadoop3/python/lib/pyspark.zip/pyspark/daemon.py", line 200, in manager
BrokenPipeError: [Errno 32] Broken pipe


In [39]:
df.write.mode('overwrite').csv('../ratings_netlfix')